# IWV

In [1]:
from metacatalog import api, models
import pandas as pd
from collections import defaultdict
from datetime import datetime as dt
from datetime import timedelta as td

In [2]:
session = api.connect_database()

## Metadata

In [3]:
# Author
author = api.find_person(session, last_name='Yuan', return_iterator=True).first()
if author is None:
    author = api.add_person(
        session, 
        first_name='Peng',
        last_name='Yuan', 
        affiliation='Geodetic Institute (GIK), KIT',
        organisation_name='Karlsruhe Institute for Technology (KIT)',
        organisation_abbrev='KIT'
    )

print(author)

Peng Yuan <ID=6>


In [4]:
# variables
iwv = api.find_variable(session, symbol='iwv', return_iterator=True).first()
if iwv is None:
    unit = api.add_unit(session, name='Kilogram per squaremeter', symbol='kg*m^-2', si='kg*m^-2')
    iwv = api.add_variable(
        session,
        name='Integrated Water Vapor',
        symbol='iwv',
        column_names=['ivw'],
        unit=unit.id
    )
print(iwv)

# air temperature
temp = api.find_variable(session, name='air temperature', return_iterator=True).one()
print(temp)

press = api.find_variable(session, name='air pressure', return_iterator=True).one()
print(press)

Integrated Water Vapor [kg*m^-2] <ID=10005>
air temperature [C] <ID=1>
air pressure [10^2*Pa] <ID=5>


In [5]:
# Abstract
abstract = """
We obtained the one hourly time series of Integrated Water Vapor (IWV) for 66 stations of the GNSS Upper Rhine Graben network (GURN) network.
The time period is from January 1, 2015 to June 30, 2019.
The IWV were calculated with tropospheric Zenith Total Delay (ZTD) from Global Positioning System (GPS) at the stations.
Station pressure (Ps) and weighte mean temperature (Tm) were also computed from ERA5 reanalysis for the retieval of the GPS IWV.
"""

In [6]:
# lincenses
for lic in api.find_license(session):
    print(lic)
    
license = api.find_license(session, id=5)[0]
print('\n---------------\n', license)

Open Data Commons Open Database License <ID=4>
Open Data Commons Attribution License v1.0 <ID=5>
Creative Commons Attribution 4.0 International <ID=6>
Creative Commons Attribution-ShareAlike 4.0 International <ID=7>
Creative Commons Attribution-NonCommerical 4.0 International <ID=8>
Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International <ID=9>
Data licence Germany – attribution – version 2.0 <ID=10>

---------------
 Open Data Commons Attribution License v1.0 <ID=5>


In [7]:
# load the datafile
with open('./gik150010.txt', 'r') as f:
    raw = f.read()

# extract the stations
stations = defaultdict(lambda: dict())
iwv = defaultdict(lambda: [])

current_block = None
for line in raw.splitlines():
    # Check for start blocks
    if '+SITE/ID' in line:
        current_block = 'site'
        continue
    if '+SITE/COORDINATES' in line:
        current_block = 'coords'
        continue
    if '+SITE/ECCENTRICITY' in line:
        current_block = 'ccentricity'
        continue
    if '+SITE/RECEIVER' in line:
        current_block = 'receiver'
        continue

    if '+SITE/ANTENNA' in line:
        current_block = 'antenna'
        continue

    if '+TROP/SOLUTION' in line:
        current_block = 'data'
        continue

    # check for end blocks
    if '-SITE/ID' in line or '-SITE/COORDINATES' in line or '-SITE/ECCENTRICITY' in line or '-SITE/RECEIVER' in line or '-SITE/ANTENNA' in line or '-TROP/SOLUTION' in line:
        current_block = None
        continue

    # we do not process headers for now. They are hardcoded
    if line.startswith('*'):
        continue

    # process the blocks
    if current_block == 'site':
        chunks = line.split()
        stations[chunks[0]] = dict(location=f"SRID=4327;POINT({chunks[3]} {chunks[4]})", height=float(chunks[6]))
    
    if current_block == 'coords':
        c = line.split()
        stations[c[0]].update(dict(x=float(c[6]), y=float(c[7]), z=float(c[8]), system=c[9]))
    
    if current_block == 'ccentricity':
        c = line.split()
        stations[c[0]].update({'eccentricity': dict(up=float(c[7]), north=float(c[8]), east=float(c[9]))})
    
    if current_block == 'receiver':
        c = line.split()
        stations[c[0]].update({'receiver': dict(description=f"{c[6]} {c[7]}", s_n=c[8])})

    if current_block == 'antenna':
        c = line.split()
        stations[c[0]].update({'antenna': dict(description=f"{c[6]} {c[7]}", s_n=c[8])})

    if current_block == 'data':
        c = line.split()

        # create date
        tc = c[1].split(':')

        date = dt(int(tc[0]), 1, 1) + td(days = int(tc[1]) - 1) + td(seconds=int(tc[2]))

        # TROTOT STDDEV TGNTOT STDDEV TGETOT STDDEV PRESS_ TRODRY TROWET WMTEMP IWV___
        iwv[c[0]].append({
            'tstamp': date,
            'iwv': float(c[10]),
            'trotot': float(c[2]),
            'trotot_stddev': float(c[3]),
            'tgetot': float(c[4]),
            'tgetot_stddev': float(c[5]),
            'press': float(c[6]),
            'trodry': float(c[7]),
            'trowet': float(c[8]),
            'wmtemp': float(c[9])

        })

stations[list(stations.keys())[0]]
iwv[list(iwv.keys())[0]]

[{'tstamp': datetime.datetime(2015, 1, 1, 0, 0),
  'iwv': 94.26,
  'trotot': 2371.4,
  'trotot_stddev': 7.7,
  'tgetot': 1.7,
  'tgetot_stddev': 7.9,
  'press': 1.5,
  'trodry': 7.1,
  'trowet': 1000.4,
  'wmtemp': 2277.1},
 {'tstamp': datetime.datetime(2015, 1, 1, 1, 0),
  'iwv': 97.76,
  'trotot': 2375.2,
  'trotot_stddev': 3.6,
  'tgetot': 0.91,
  'tgetot_stddev': 7.86,
  'press': 1.48,
  'trodry': 7.07,
  'trowet': 1000.5,
  'wmtemp': 2277.4},
 {'tstamp': datetime.datetime(2015, 1, 1, 2, 0),
  'iwv': 97.59,
  'trotot': 2375.0,
  'trotot_stddev': 2.8,
  'tgetot': 0.13,
  'tgetot_stddev': 7.83,
  'press': 1.45,
  'trodry': 7.04,
  'trowet': 1000.5,
  'wmtemp': 2277.4},
 {'tstamp': datetime.datetime(2015, 1, 1, 3, 0),
  'iwv': 100.62,
  'trotot': 2377.9,
  'trotot_stddev': 3.3,
  'tgetot': -0.66,
  'tgetot_stddev': 7.79,
  'press': 1.43,
  'trodry': 7.02,
  'trowet': 1000.5,
  'wmtemp': 2277.3},
 {'tstamp': datetime.datetime(2015, 1, 1, 4, 0),
  'iwv': 101.14,
  'trotot': 2378.0,
  't

### build the entries and composite group

In [1]:
title = 'GPS %s product Station %s of the GURN network'
all_entries = []

for stat_id, meta in stations.items():
    gr_entries = []

    # iwv
    iwv_en = api.find_entry(session, title=title % ('IWV', stat_id), author=author, return_iterator=True).first()
    if iwv_en is None:
        iwv_en: models.Entry = api.add_entry(
            session,
            title=title % ('IWV', stat_id),
            author=author.id,
            loaction=meta['location'],
            variable=iwv.id,
            abstract=abstract,
            license=license,
            embargo=False
        )
        iwv_en.add_details({k: v for k, v in meta.items() if k != 'location'}, commit=True)
        #iwv_en.create_datasource()
        #iwv_en.import_data()
    gr_entries.append(iwv_en)

    # press
    pre_en = api.find_entry(session, title=title % ('athmospheric pressure', stat_id), author=author, return_iterator=True).first()
    if pre_en is None:
        pre_en: models.Entry = api.add_entry(
            session,
            title=title % ('athmosperic pressure', stat_id),
            author=author.id,
            loaction=meta['location'],
            variable=press.id,
            abstract=abstract,
            license=license,
            embargo=False
        )
        pre_en.add_details({k: v for k, v in meta.items() if k != 'location'}, commit=True)
    gr_entries.append(pre_en)

    # partial entry
    par_en = api.find_entry(session, title=title % ('additional information', stat_id), author=author, return_iterator=True).first()
    if par_en is None:
        par_en: models.Entry = api.add_entry(
            session,
            title=title % ('athmosperic pressure', stat_id),
            author=author.id,
            loaction=meta['location'],
            variable=press.id,
            abstract=abstract,
            license=license,
            embargo=False,
            is_partial=True
        )

        par_en.add_details({k: v for k, v in meta.items() if k != 'location'}, commit=True)
    gr_entries.append(par_en)

    # create the composite
    composite = iwv_en.make_composite(others=[par_en, pre_en], title='GPS IWS product Station %s of the GURN network' % stat_id, description=abstract, commit=True)
    all_entries.extend(gr_entries)


NameError: name 'stations' is not defined